# Word count with PySpark

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* The EMR cluster attached to this notebook should have the `Spark` application installed.
* This example requires the user to create a test file on the EMR cluster master node and copy it to hdfs. Follow the `Setup` steps.
* This notebook uses the `PySpark` kernel.
***

## Introduction
In this example we use pyspark to count the occurrence of each word in the file stored in hdfs and store the results to s3.
***

## Setup
1. Create a S3 bucket location to save your results. For example: s3://EXAMPLE-BUCKET/word-count/

2. Connect to the master node of the EMR cluster using SSH. Create a test file on the master node of the EMR cluster that you wish to perform the word count on. Copy the test file from the master node to HDFS as shown in the following example.

```
 hdfs dfs -copyFromLocal test_file.txt 
```
***

## Example

Read the test file from hdfs.

In [ ]:
data = sc.textFile("hdfs:///user/hadoop/test_file.txt") # Change this to the test_file.txt that you created in hdfs in Step 2.

Display the contents of test file.

In [ ]:
data.collect()

Count the occurance of each word.

In [ ]:
counts = data.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a+b)

Display the count for each word.

In [ ]:
counts.collect()

Save the results to your s3 bucket.

In [ ]:
counts.saveAsTextFile("s3://EXAMPLE-BUCKET/word-count") # Change this to the S3 location that you created in Step 1.